In [ ]:
import torch
# Load model with pretrained weights
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get(Models.PP_YOLOE_S, pretrained_weights="coco")

# Prepare model for conversion
# Input size is in format of [Batch x Channels x Width x Height] where 640 is the standard COCO dataset dimensions

model.eval()
    
# Create dummy_input
dummy_input = torch.rand(size=[1, 3, 640, 640])

# Convert model to onnx
torch.onnx.export(model, dummy_input,  "/workspace/pp-yoloe-s-custom-sg.onnx")

In [ ]:
model.predict('/datasets/cat/images/IMG_20210726_161004.jpg').show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image_path = '/datasets/cat/images/IMG_20210726_161004.jpg'
image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
def preprocess_image(image, size=640):
    # resize
    image = cv2.resize(image, dsize=[size,size])
    # standardize
    image = image.astype(np.float32) / 255.0
    # normalize, COCO mean and std
    image = (image - np.array([123.675, 116.28, 103.53])) / np.array([58.395,  57.12,  57.375])
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = np.transpose( image, (2, 0, 1))
    image = np.expand_dims(image, axis=0)
    return torch.from_numpy(image.astype('float32'))

In [ ]:
pip_out = model._get_pipeline()(image)
bbox = pip_out[0].prediction.bboxes_xyxy
confidence = pip_out[0].prediction.confidence
labels = pip_out[0].prediction.labels

In [ ]:
p1 = bbox[0][:2].astype('int')
p2 = bbox[0][2:].astype('int')
cv2.rectangle(image, pt1=p1, pt2=p2, color=(0, 255, 0), thickness=2)

plt.imshow(image)
plt.show()

In [ ]:
input_image = preprocess_image(image)

# input_image = model._get_pipeline()(image)

model_output = model.forward( input_image )
predictions = model_output[0]


cls_thrs = 0.5
nms_iout = 0.7

for pred_bboxes, pred_scores in zip(*predictions):
    # pred_bboxes [Anchors, 4],
    # pred_scores [Anchors, C]
    print(pred_bboxes.shape)
    print(pred_scores.shape)

    # filter by score_thrs
    pred_cls_conf, pred_cls_label = torch.max(pred_scores, dim=1)
    print(pred_cls_conf)

In [ ]:
# import onnx

# onnx_model = onnx.load("/workspace/pp-yoloe-s-custom-sg.onnx")
# onnx.checker.check_model(onnx_model)

In [ ]:
# x = dummy_input
# torch_out = model(x)

In [ ]:
# import onnxruntime
# import numpy as np

# ort_session = onnxruntime.InferenceSession("/workspace/pp-yoloe-s-custom-sg.onnx")

# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# # compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
# ort_outs = ort_session.run(None, ort_inputs)

# # compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

# print("Exported model has been tested with ONNXRuntime, and the result looks good!")